# Baseline Veeva Vault API Testing Template

This is a minimal baseline template for Veeva Vault API testing that contains only the essential components needed by all section-specific test notebooks.

## Template Purpose

This template should be **copied and customized** for testing each of the 38 API documentation sections:

**Core APIs (01-10):**
- 01_authentication.md → 01_AuthenticationTests.ipynb
- 02_direct_data.md → 02_DirectDataTests.ipynb  
- 03_vault_query_language_vql.md → 03_VQLTests.ipynb
- 04_metadata_definition_language_mdl.md → 04_MDLTests.ipynb
- 05_documents.md → 05_DocumentTests.ipynb
- 06_binders.md → 06_BinderTests.ipynb
- 07_vault_objects.md → 07_ObjectTests.ipynb
- 08_document_and_binder_roles.md → 08_RoleTests.ipynb
- 09_workflows.md → 09_WorkflowTests.ipynb
- 10_document_lifecycle_workflows.md → 10_DocLifecycleTests.ipynb

**And 28 more sections...**

## What This Template Provides

✅ **Essential imports** - VaultClient, Authentication, Query services  
✅ **Credential loading** - from test_credentials.py  
✅ **Authentication framework** - with session management  
✅ **Test result tracking** - structured logging and timing  
✅ **Error handling** - comprehensive exception management  

## What to Add in Section-Specific Notebooks

➕ **Additional service imports** - specific to the API section  
➕ **Section-specific tests** - based on api_docs/sections/*.md  
➕ **Endpoint validation** - following notebook_tests/*.md  
➕ **Custom test cases** - for specific use cases  

## Prerequisites

- Active .venv with veevatools installed
- Valid test_credentials.py with vault access
- VeevaVault client and services properly configured

In [2]:
# Core imports
import sys
import os
import json
import pandas as pd
from datetime import datetime
from typing import Dict, Any, Optional, List
import time

# VeevaVault Core Client
from veevavault.client import VaultClient

# Essential Services for baseline template
from veevavault.services.authentication import AuthenticationService
from veevavault.services.queries import QueryService

print("✅ Successfully imported essential VeevaVault services")
print(f"🐍 Python version: {sys.version}")
print(f"📦 Virtual environment active: {sys.prefix != sys.base_prefix}")
print("📚 Baseline template loaded with core services only")

✅ Successfully imported essential VeevaVault services
🐍 Python version: 3.11.11 (main, Jan 30 2025, 20:55:05) [Clang 16.0.0 (clang-1600.0.26.6)]
📦 Virtual environment active: True
📚 Baseline template loaded with core services only


## Configuration and Credentials

Load vault credentials and configure the testing environment.

**Required:** Ensure `test_credentials.py` exists with valid vault access credentials.

In [3]:
# Import credentials from secure configuration
try:
    from test_credentials import TEST_VAULTS, DEFAULT_VAULT
    
    # Get the default vault credentials
    vault_config = TEST_VAULTS[DEFAULT_VAULT]
    
    # Extract credentials
    VAULT_URL = vault_config["URL"]
    VAULT_USERNAME = vault_config["username"] 
    VAULT_PASSWORD = vault_config["password"]
    VAULT_DNS = VAULT_URL.replace("https://", "").replace("http://", "").rstrip("/")
    
    print("✅ Successfully imported credentials from test_credentials.py")
    print(f"🏢 Using vault: {DEFAULT_VAULT}")
    print(f"📡 Vault DNS: {VAULT_DNS}")
    print(f"👤 Username: {VAULT_USERNAME}")
    print(f"🔗 Full URL: {VAULT_URL}")
    
except ImportError as e:
    print("❌ Failed to import credentials!")
    print("Please ensure test_credentials.py exists and contains:")
    print("- TEST_VAULTS dictionary")
    print("- DEFAULT_VAULT setting")
    raise e
except KeyError as e:
    print(f"❌ Configuration error: {e}")
    print(f"Available vaults: {list(TEST_VAULTS.keys()) if 'TEST_VAULTS' in locals() else 'None'}")
    raise e

✅ Successfully imported credentials from test_credentials.py
🏢 Using vault: michael_mastermind
📡 Vault DNS: vv-consulting-michael-mastermind.veevavault.com
👤 Username: veevatools@vv-consulting.com
🔗 Full URL: https://vv-consulting-michael-mastermind.veevavault.com/


## Baseline Testing Framework

Minimal testing framework with essential components for all API testing.

In [4]:
class APITestResult:
    """Structured container for API test results."""
    
    def __init__(self, test_name: str, status: str, section: str = None, 
                 endpoint: str = None, details: Dict[str, Any] = None, 
                 error: str = None, response_data: Any = None,
                 execution_time: float = None):
        self.timestamp = datetime.now().isoformat()
        self.test_name = test_name
        self.status = status  # SUCCESS, FAILURE, WARNING, SKIPPED
        self.section = section  # API section (01-38)
        self.endpoint = endpoint  # Specific API endpoint
        self.details = details or {}
        self.error = error
        self.response_data = response_data
        self.execution_time = execution_time

class BaselineAPITester:
    """Minimal baseline API testing framework."""
    
    def __init__(self, vault_client: VaultClient):
        self.vault_client = vault_client
        self.test_results: List[APITestResult] = []
        self.session_active = False
        
        # Initialize essential services
        self.auth_service = AuthenticationService(vault_client)
        self.query_service = QueryService(vault_client)
        
    def log_test_result(self, test_name: str, status: str, section: str = None,
                       endpoint: str = None, details: Dict[str, Any] = None,
                       error: str = None, response_data: Any = None,
                       execution_time: float = None):
        """Log a test result."""
        result = APITestResult(
            test_name=test_name, status=status, section=section, endpoint=endpoint,
            details=details, error=error, response_data=response_data, execution_time=execution_time
        )
        self.test_results.append(result)
        
        # Print status with emoji
        status_emoji = {'SUCCESS': '✅', 'FAILURE': '❌', 'WARNING': '⚠️', 'SKIPPED': '⏭️'}
        section_info = f" [{section}]" if section else ""
        time_info = f" ({execution_time:.2f}s)" if execution_time else ""
        
        print(f"{status_emoji.get(status, '❓')} {test_name}{section_info}: {status}{time_info}")
        if error:
            print(f"   ❌ Error: {error}")
        if details:
            print(f"   📋 Details: {details}")
    
    def test_with_timing(self, test_func, test_name: str, section: str = None, 
                        endpoint: str = None, **kwargs):
        """Execute a test function with timing and error handling."""
        start_time = time.time()
        
        try:
            result = test_func(**kwargs)
            execution_time = time.time() - start_time
            
            if isinstance(result, dict) and result.get('responseStatus') == 'SUCCESS':
                self.log_test_result(test_name, 'SUCCESS', section, endpoint, 
                                   response_data=result, execution_time=execution_time)
            elif isinstance(result, dict) and result.get('responseStatus') == 'FAILURE':
                error_msg = result.get('errors', [{}])[0].get('message', 'API returned FAILURE')
                self.log_test_result(test_name, 'FAILURE', section, endpoint, 
                                   error=error_msg, response_data=result, execution_time=execution_time)
            else:
                self.log_test_result(test_name, 'SUCCESS', section, endpoint, 
                                   response_data=result, execution_time=execution_time)
            return result
            
        except Exception as e:
            execution_time = time.time() - start_time
            self.log_test_result(test_name, 'FAILURE', section, endpoint, 
                               error=str(e), execution_time=execution_time)
            return None
    
    def authenticate(self) -> bool:
        """Perform authentication and setup session."""
        auth_response = self.test_with_timing(
            test_func=self._do_authentication,
            test_name="Authentication",
            section="01",
            endpoint="/api/auth"
        )
        return auth_response is not None and auth_response.get('responseStatus') == 'SUCCESS'
    
    def _do_authentication(self):
        """Internal authentication implementation."""
        auth_response = self.auth_service.authenticate_with_username_password(
            username=VAULT_USERNAME,
            password=VAULT_PASSWORD,
            vaultDNS=VAULT_DNS
        )
        
        if auth_response.get('responseStatus') == 'SUCCESS':
            session_id = auth_response.get('sessionId')
            user_id = auth_response.get('userId')
            vault_id = auth_response.get('vaultId')
            
            # Update client with session info
            self.vault_client.sessionId = session_id
            self.vault_client.session_id = session_id
            self.vault_client.vaultId = vault_id
            self.vault_client.vaultDNS = VAULT_DNS
            
            self.session_active = True
        else:
            self.session_active = False
            
        return auth_response
    
    def print_summary(self):
        """Print test summary."""
        if not self.test_results:
            print("No tests executed yet.")
            return
            
        df = pd.DataFrame([{
            'test_name': r.test_name,
            'status': r.status,
            'section': r.section,
            'execution_time': r.execution_time
        } for r in self.test_results])
        
        status_counts = df['status'].value_counts()
        print("\n📊 TEST SUMMARY")
        print("=" * 40)
        for status, count in status_counts.items():
            emoji = {'SUCCESS': '✅', 'FAILURE': '❌', 'WARNING': '⚠️', 'SKIPPED': '⏭️'}.get(status, '❓')
            print(f"{emoji} {status}: {count}")
        
        if 'execution_time' in df.columns and df['execution_time'].notna().any():
            print(f"⏱️ Total time: {df['execution_time'].sum():.2f}s")
        print(f"🔐 Session: {'✅ Active' if self.session_active else '❌ Inactive'}")

print("✅ Baseline API testing framework initialized")

✅ Baseline API testing framework initialized


## Client Initialization and Authentication

Initialize the VaultClient and perform authentication to establish the testing session.

In [5]:
# Initialize the main vault client
print("🔧 Initializing VeevaVault client...")

vault_client = VaultClient()
vault_client.vaultURL = VAULT_URL
vault_client.vaultDNS = VAULT_DNS
vault_client.vaultUserName = VAULT_USERNAME
vault_client.vaultPassword = VAULT_PASSWORD

print(f"✅ VeevaVault client initialized")
print(f"🔗 Vault URL: {vault_client.vaultURL}")
print(f"📡 Vault DNS: {vault_client.vaultDNS}")
print(f"👤 Username: {vault_client.vaultUserName}")

# Initialize the baseline test framework
print("🔧 Initializing baseline test framework...")
baseline_tester = BaselineAPITester(vault_client)

print(f"✅ Baseline framework ready")

🔧 Initializing VeevaVault client...
✅ VeevaVault client initialized
🔗 Vault URL: https://vv-consulting-michael-mastermind.veevavault.com/
📡 Vault DNS: vv-consulting-michael-mastermind.veevavault.com
👤 Username: veevatools@vv-consulting.com
🔧 Initializing baseline test framework...
✅ Baseline framework ready


In [6]:
# Perform authentication 
print("🔐 Performing authentication...")

auth_success = baseline_tester.authenticate()

if auth_success:
    print(f"\n🎉 Authentication Successful!")
    print(f"🆔 Session ID: {vault_client.sessionId[:20]}..." if vault_client.sessionId else "No session ID")
    print(f"🏢 Vault ID: {vault_client.vaultId}")
    print("✅ Baseline template validated - ready for section-specific testing!")
else:
    print("❌ Authentication failed! Check credentials and try again.")

# Show summary
baseline_tester.print_summary()

🔐 Performing authentication...
✅ Authentication [01]: SUCCESS (0.50s)

🎉 Authentication Successful!
🆔 Session ID: EFFDF861DBECA76326C1...
🏢 Vault ID: 92425
✅ Baseline template validated - ready for section-specific testing!

📊 TEST SUMMARY
✅ SUCCESS: 1
⏱️ Total time: 0.50s
🔐 Session: ✅ Active
✅ Authentication [01]: SUCCESS (0.50s)

🎉 Authentication Successful!
🆔 Session ID: EFFDF861DBECA76326C1...
🏢 Vault ID: 92425
✅ Baseline template validated - ready for section-specific testing!

📊 TEST SUMMARY
✅ SUCCESS: 1
⏱️ Total time: 0.50s
🔐 Session: ✅ Active


## Session Management and Next Steps

Basic session utilities and guidance for creating section-specific test notebooks.

### Available Functions:
- `get_session_info()` - Display current session status
- `end_session()` - Clean session logout
- `baseline_tester.print_summary()` - Show test results

### Create Section-Specific Notebooks:

1. **Copy this baseline template** 
2. **Import additional services** for the specific API section
3. **Add test methods** based on `api_docs/sections/XX_*.md`
4. **Follow test plans** in `notebook_tests/XX_*.md`
5. **Run comprehensive testing** for the section

**✅ Baseline template validated and ready for use!**

In [8]:
# Session management utilities
def get_session_info():
    """Get current session information."""
    print("🔍 SESSION INFO")
    print("="*30)
    print(f"Active: {'✅ Yes' if baseline_tester.session_active else '❌ No'}")
    print(f"Session ID: {vault_client.sessionId[:20] + '...' if vault_client.sessionId else 'None'}")
    print(f"Vault ID: {vault_client.vaultId or 'None'}")
    print(f"Tests Run: {len(baseline_tester.test_results)}")

def end_session():
    """End the current session."""
    if hasattr(baseline_tester.auth_service, 'logout') and baseline_tester.session_active:
        try:
            baseline_tester.auth_service.logout()
            baseline_tester.session_active = False
            vault_client.sessionId = None
            print("✅ Session ended")
        except Exception as e:
            print(f"❌ Logout failed: {str(e)}")
    else:
        print("⚠️ No active session or logout not available")

print("✅ Session utilities ready: get_session_info(), end_session()")

# Display current session
get_session_info()

✅ Session utilities ready: get_session_info(), end_session()
🔍 SESSION INFO
Active: ✅ Yes
Session ID: EFFDF861DBECA76326C1...
Vault ID: 92425
Tests Run: 1
